> **Note:** This notebook is under construction. Please check back later for more updates.

# Advanced Usage
With the new version, you can now sign in to your account and use the api to its full potential.
For now the only provider supported is email, but more will be added in the future.
```python
from perplexity import Perplexity

perplexity = Perplexity("example@email.com")
```
And then you will receive an email from Perplexity AI. Copy the link associated with the `Sign in` button in the middle of the email.
The program will create a new file: `.perplexity_session` for keeping the session cookies.


Основываясь на новой версии, вы можете войти в свой аккаунт и использовать api её на полную мощность. Но надо убрать ручную регистрацию. Для этого надо сделать клиент, который будет проверять почточный ящик на наличие письма с ссылкой для входа. После этого надо будет перейти по ссылке и получить куки.

Для создания скрипта, который будет логиниться в Gmail и проверять URL от helpdesk'а с коллбэком, можно использовать Python и библиотеку `imaplib`. Вам нужно будет включить доступ по IMAP в настройках вашего аккаунта Gmail и, возможно, создать специальный пароль приложения, если у вас включена двухфакторная аутентификация[2][6][14].

Вот пример скрипта на Python, который логинится в Gmail, ищет непрочитанные письма и извлекает из них URL:

```python
import imaplib
import email
from email.header import decode_header
import re

# учетные данные для входа в Gmail
username = "your_email@gmail.com"
password = "your_password"

# создаем экземпляр IMAP4_SSL класса
mail = imaplib.IMAP4_SSL("imap.gmail.com")

# аутентификация
mail.login(username, password)

# выбираем почтовый ящик (в данном случае "inbox")
mail.select("inbox")

# ищем письма (все непрочитанные письма)
status, messages = mail.search(None, 'UNSEEN')

# преобразуем messages в список ID писем
messages = messages[0].split(b' ')

# регулярное выражение для поиска URL
url_pattern = re.compile(r'https?://\S+')

# проходимся по каждому письму
for mail_id in messages:
    # получаем данные письма
    _, msg = mail.fetch(mail_id, '(RFC822)')

    # парсим данные письма
    for response in msg:
        if isinstance(response, tuple):
            # парсим байтовый объект письма
            msg = email.message_from_bytes(response[1])
            
            # декодируем тему письма
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding)
            
            # проверяем, содержит ли тема письма ключевое слово для коллбэка
            if "callback" in subject.lower():
                # если содержит, то извлекаем URL
                if msg.is_multipart():
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        if content_type == "text/html":
                            body = part.get_payload(decode=True).decode()
                            # ищем URL в теле письма
                            urls = url_pattern.findall(body)
                            for url in urls:
                                print(url)
                else:
                    # если письмо не состоит из нескольких частей
                    content_type = msg.get_content_type()
                    if content_type == "text/html":
                        body = msg.get_payload(decode=True).decode()
                        # ищем URL в теле письма
                        urls = url_pattern.findall(body)
                        for url in urls:
                            print(url)

# закрываем соединение и выходим
mail.close()
mail.logout()
```

Этот скрипт использует регулярное выражение для поиска URL в HTML-содержимом письма. Обратите внимание, что вам нужно будет заменить `"your_email@gmail.com"` и `"your_password"` на ваши реальные учетные данные Gmail. Также, если в письме используется другой тип контента (не "text/html"), вам нужно будет соответствующим образом изменить скрипт.

Пожалуйста, убедитесь, что вы следуете всем требованиям безопасности и конфиденциальности при работе с электронной почтой и личными данными.

Citations:
[1] https://github.com/nathanrchn/perplexityai
[2] https://support.google.com/mail/thread/184123470/cannot-login-to-gmail-through-imap-in-python-something-to-do-with-less-secure-apps-update?hl=en
[3] https://stackoverflow.com/questions/1792366/extract-urls-out-of-email-in-python
[4] https://stackoverflow.com/questions/3362600/how-to-send-email-attachments
[5] https://stackoverflow.com/questions/38417077/how-to-detect-url-callback-request
[6] https://stackoverflow.com/questions/70188644/gmail-login-error-with-the-help-of-imaplib-in-python
[7] https://youtube.com/watch?v=u3XVKM3ZgNg
[8] https://realpython.com/python-send-email/
[9] https://blog.perplexity.ai/blog/introducing-pplx-api
[10] https://docs.python.org/3/library/imaplib.html
[11] https://www.geeksforgeeks.org/python-extract-domain-name-from-email-address/
[12] https://betterprogramming.pub/how-to-send-an-email-with-attachments-in-python-abe3b957ecf3?gi=2d1c686f6691
[13] https://pplx-res.cloudinary.com/image/upload/t_fit/v1703496779/user_uploads/nsxalkcvv/image.jpg
[14] https://www.reddit.com/r/learnprogramming/comments/unzzaq/how_to_add_gmail_imap_authentication_python/?rdt=58841
[15] https://forum.uipath.com/t/get-hyperlink-address-from-an-email-body/6060
[16] https://pythonalgos.com/2021/11/02/how-to-send-an-email-with-an-attachment-in-python/
[17] https://gist.github.com/jasonrdsouza/1674794
[18] https://www.tutorialspoint.com/python_text_processing/python_extract_url_from_text.htm
[19] https://mailtrap.io/blog/python-send-email/
[20] https://gist.github.com/morenoh149/f2ed47dc8c57bacf5bc27a899c642d6a
[21] https://forum.keyboardmaestro.com/t/how-do-i-extract-urls-from-emails-and-open-in-selected-browser/16709
[22] https://youtube.com/watch?v=V6hQfGxlPgw
[23] http://www.learningaboutelectronics.com/Articles/How-to-access-IMAP-in-gmail-with-Python.php
[24] https://faun.pub/extract-all-website-links-in-python-48f07619db95?gi=51c6dd345f66
[25] https://discuss.python.org/t/extracting-and-saving-an-email-pdf-attachment/33016


import imaplib
import email
from email.header import decode_header
import re

# учетные данные для входа в Gmail
username = "ormwish@gmail.com"
password = getpass.getpass("Password: ")

# создаем экземпляр IMAP4_SSL класса
mail = imaplib.IMAP4_SSL("imap.gmail.com")

# аутентификация
mail.login(username, password)

# выбираем почтовый ящик (в данном случае "inbox")
mail.select("inbox")

# ищем письма (все непрочитанные письма)
status, messages = mail.search(None, 'UNSEEN')

# преобразуем messages в список ID писем
messages = messages[0].split(b' ')

# регулярное выражение для поиска URL
url_pattern = re.compile(r'https?://\S+')

# проходимся по каждому письму
for mail_id in messages:
    # получаем данные письма
    _, msg = mail.fetch(mail_id, '(RFC822)')

    # парсим данные письма
    for response in msg:
        if isinstance(response, tuple):
            # парсим байтовый объект письма
            msg = email.message_from_bytes(response[1])
            
            # декодируем тему письма
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding)
            
            # проверяем, содержит ли тема письма ключевое слово для коллбэка
            if "callback" in subject.lower():
                # если содержит, то извлекаем URL
                if msg.is_multipart():
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        if content_type == "text/html":
                            body = part.get_payload(decode=True).decode()
                            # ищем URL в теле письма
                            urls = url_pattern.findall(body)
                            for url in urls:
                                print(url)
                else:
                    # если письмо не состоит из нескольких частей
                    content_type = msg.get_content_type()
                    if content_type == "text/html":
                        body = msg.get_payload(decode=True).decode()
                        # ищем URL в теле письма
                        urls = url_pattern.findall(body)
                        for url in urls:
                            print(url)

# закрываем соединение и выходим
mail.close()
mail.logout()

In [8]:
%pip install simplegmail


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 542.2 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
"""
This cell is to implement the GMAIL Client and function for extracting URLs from email body.
"""
from devtools import debug
from simplegmail import Gmail
import re
GMAIL_CLIENT_SECRET = "client_secret.json"
client: Gmail = Gmail(client_secret_file = GMAIL_CLIENT_SECRET)

# Функция для поиска URL в тексте
def extract_urls(data:str) -> list[str]:
    # Регулярное выражение для поиска URL
    regex = r"https?://[^\s]+"
    urls = re.findall(regex, data)
    return urls

# Получение всех писем от helpdesk.perplexity.ai
messages = client.get_messages(query="from:helpdesk.perplexity.ai")

last_message = messages[0]
urls = extract_urls(last_message.html)
cburl = urls[2]
print(cburl)

In [32]:
print(cburl)

https://www.perplexity.ai/api/auth/callback/email?callbackUrl=https%3A%2F%2Fwww.perplexity.ai&amp;token=861d89f5ea3eec5bf30625047983140e1f7bb40567ac8c398a7dce8fbe74b07a&amp;email=ormwish%40gmail.com"


>Как показывает результат 32 ячейки -- ссылка на коллбек авторизации содержит в себе лишнюю кавычку. Надо её убрать.
> ## Task
> 1. Убрать лишнюю кавычку из ссылки на коллбек авторизации.
> 2. Для этого нужно переработать функцию `extract_urls` так, чтобы она возвращала чистые URL без кавычек и прочего мусора.